In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import networkx as nx
import numpy as np
np.set_printoptions(precision=3)

import pandas as pd

from TCGAMultiOmics.multiomics import MultiOmicsData

from moge.network.heterogeneous_network import HeterogeneousNetwork
from moge.network.omics_distance import compute_expression_correlations, compute_annotation_similarity
# from moge.visualization. import matrix_heatmap

# Heterogeneous Network

In [2]:
# READ Edgeslist
# network.import_edgelist_file(file="moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_node_similarity_99_50.edgelist", 
#                              directed=False)
# network.import_edgelist_file(file="moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_regulatory_interactions.edgelist", 
#                              directed=True)
# network.remove_extra_nodes()

In [3]:
# WRITE Edgelist
# nx.write_edgelist(network_train.get_subgraph(["MIR", "GE", "LNC"]), 
#                   "moge/data/lncRNA_miRNA_mRNA/lncRNA-miRNA-mRNA_network_lncbase_mi-lnc_filtered.edgelist", 
#                   data=False)

In [4]:
import pickle



# READ
with open('moge/data/lncbase_mi-lnc/lncRNA-miRNA-mRNA_network_lncbase_mi-lnc_filtered.pickle', 'rb') as file:
    network = pickle.load(file)


#  Split Network to train test and write to file

In [5]:
from moge.evaluation.utils import split_train_test_edges

network_train, val_edges_dict, test_edges_dict = split_train_test_edges(network, network.node_list, 
                                                          test_frac=.01, val_frac=.30, edge_types=["u", "d", "u_n"], 
                                                          seed=0)

Removed 3276 test, and 98285 val, type u edges
Removed 4891 test, and 146735 val, type d edges
Removed 4861 test, and 145842 val, type u_n edges


In [5]:
from moge.evaluation.utils import split_train_test_nodes

network_train, val_edges_dict, test_edges_dict = split_train_test_nodes(network, network.node_list, 
                                                          test_frac=.01, val_frac=.10, edge_types=["u", "u_n"], 
                                                                        seed=0)

preprocessing...
removed 188269 edges, and  341 nodes.
validation edges [('u', 17201), ('u_n', 56259)]
test edges [('u', 1714), ('u_n', 0)]


In [7]:
network_train.G.number_of_edges()

985894

In [8]:
network.node_list.__len__()

17887

# Create a new network with test edges

In [6]:
from moge.network.heterogeneous_network import HeterogeneousNetwork

network_test = HeterogeneousNetwork(modalities=["MIR", "GE", "LNC"], multi_omics_data=network.multi_omics_data)
network_val = HeterogeneousNetwork(modalities=["MIR", "GE", "LNC"], multi_omics_data=network.multi_omics_data)
node_list = network_train.node_list

MIR  nodes: 1870
GE  nodes: 20284
LNC  nodes: 12725
Total nodes: 34879
MIR  nodes: 1870
GE  nodes: 20284
LNC  nodes: 12725
Total nodes: 34879


In [7]:
for edge_type, edges in test_edges_dict.items():
    for edge in edges:
#         network_test.G.add_edge(node_list[edge[0]], node_list[edge[1]], type=edge_type) # for split edges
        network_test.G.add_edge(edge[0], edge[1], type=edge_type) # for split nodes
print(network_test.G.number_of_edges())

1714


In [8]:
for edge_type, edges in val_edges_dict.items():
    for edge in edges:
#         network_val.G.add_edge(node_list[edge[0]], node_list[edge[1]], type=edge_type) # for split edges
        network_val.G.add_edge(edge[0], edge[1], type=edge_type) # For split nodes
print(network_val.G.number_of_edges())

73460


# Write train and test networks to file

In [9]:
# WRITE
with open('moge/data/lncbase_filtered_nodes_split/lmn_net_lncbase_filtered_10_u_un_train.pickle', 'wb') as file:
    pickle.dump(network_train, file)
    
with open('moge/data/lncbase_filtered_nodes_split/lmn_net_lncbase_filtered_10_u_un_test.pickle', 'wb') as file:
    pickle.dump(network_test, file)
    
with open('moge/data/lncbase_filtered_nodes_split/lmn_net_lncbase_filtered_10_u_un_val.pickle', 'wb') as file:
    pickle.dump(network_val, file)